# Import Libraries

In [10]:
import requests
import pandas as pd
import numpy as np

import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

import seaborn as sns

# Define Functions

In [2]:
# takes a data set in wide format (dates in columns) and converts to long format (dates in rows)

def wide_to_long(df, pat1, var_nm, val_nm):

    import pandas as pd
    
    id_cols = []
    value_cols = []

    for cols in df.columns:

        try:
        
            if cols.find(pat1) == -1:
                id_cols.append(cols)

            else:

                if cols.find(pat1) != -1:
                    value_cols.append(cols)
                    
        except:
            
            value_cols.append(cols)
                
    df=(pd.melt(df, 
             id_vars=id_cols, 
             value_vars=value_cols)
       .rename(columns = {
           'variable': var_nm, 
           'value': val_nm}))

    df=df.loc[df[val_nm] > 0].reset_index()
    
    return df

In [3]:
# prepares dataframe for seasonal adjustment

def seas_prep(df,
             date_col, 
             cat_col, 
             value_col):
    
    agg_date_cols=[cat_col] 

    agg_date_cols.append(date_col)

    df_sa_dates=(df
                 .groupby(cat_col)
                 .agg({date_col: ['min', 'max']})
                 .reset_index())

    df_sa_dates.columns = ['_'.join(tup).rstrip('_') for tup in df_sa_dates.columns.values]
    
    df_sa=(df
          .groupby(agg_date_cols)[value_col]
          .sum()
          .reset_index()
          .pivot_table(index=[date_col], columns=cat_col)[value_col]
          .fillna(0)
          .stack()
          .reset_index()
          .rename(columns = {0:value_col})
          .merge(df_sa_dates))
    
    df_sa=df_sa.loc[(df_sa[date_col] >= df_sa[date_col+"_min"]) &
                    (df_sa[date_col] <= df_sa[date_col+"_max"])
                   ]
       
    return df_sa

In [4]:
# uses seasonal_decompose to create seasonal adjustment factors

def seas_exec(df_source, date_col, range_size, agg_cols, val_col):
    
    from statsmodels.tsa.seasonal import seasonal_decompose

    agg_list= df_source[agg_cols].unique()

    i = 1


    for agg in agg_list:

        df_agg_x = df_source.loc[df_source[agg_cols] == agg]

        yr_limit=(df_agg_x[date_col]
                  .sort_values(ascending = False)
                  .values[len(df_agg_x[date_col])-1]
                  .astype('datetime64[Y]')
                  .astype(int)+ 1970+range_size)

        yr_start = (df_agg_x[date_col]
                    .sort_values(ascending = False)
                    .values[0]
                    .astype('datetime64[Y]')
                    .astype(int) + 1970)

        if yr_limit <= yr_start:

            for yr in range(yr_start, yr_limit, -1):

                df_x=(df_agg_x.
                      loc[(df_agg_x.date.dt.year <= yr) & (df_agg_x.date.dt.year >= yr-range_size)]
                      .sort_values(by = date_col))
                

                if df_x.volume.min() > 0:

                    decomp=seasonal_decompose(df_x[val_col], 'multiplicative', period = 12, two_sided = True)

                    df_x['seasonal'] = decomp.seasonal


                else:

                    decomp=seasonal_decompose(df_x[val_col]+1, 'multiplicative', period = 12, two_sided = True)                
                    df_x['seasonal'] = decomp.seasonal
   

                if i == 1:

                    df_x_sa=df_x.loc[df_x.date.dt.year == yr]

                    i = 2

                else:

                    df_x_sa=df_x_sa.append(df_x.loc[df_x.date.dt.year == yr])
                    
    df_x_sa['seasonally_adjusted']=df_x_sa[val_col]/df_x_sa.seasonal
                    
    return df_x_sa.sort_values(by = date_col)

# Settings

In [5]:
website = "https://www.acea.be/uploads/statistic_documents/1990%E2%80%932019_PC_by_country_EU%2BEFTA.xlsx"
date_col = 'date'
val_col = 'volume'
cat_col = 'Country'
range_set = 4 # gives the number of years used to determime seasonality

# wrapper function

In [6]:
df=seas_exec(
    seas_prep(
        wide_to_long(
            pd.concat(
                pd.read_excel(
                    requests.get(website).content, 
                    sheet_name=None, skiprows=1, header=1), ignore_index=True)
            .drop(columns = 'FY', index=1),
            '00', date_col, val_col)
        , date_col, cat_col, val_col)
    , date_col, range_set, cat_col, val_col)

# Display Results

In [11]:
sns.set_theme()

In [14]:
def f(c):

    fig, (ax1, ax2) = plt.subplots(2,1, figsize=(14,8))
    
    x=df['date'].loc[df.Country==c]
    y=df['volume'].loc[df.Country==c]
    y2=df['seasonally_adjusted'].loc[df.Country==c]
    y3=df['seasonal'].loc[df.Country==c]
    
    fig.suptitle('Seasonal Decomposition Results')
    
    ax1.set_title('Raw Series vs. Seasonally-Adjusted Series')
    ax2.set_title('Seasonal Factors')
    
    ax1.plot(x, y,
             x, y2)
    ax2.plot(x, y3)

In [15]:
interact(f, c=sorted(df.Country.unique()))

interactive(children=(Dropdown(description='c', options=('Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Czech R…

<function __main__.f(c)>

# Output Results

In [9]:
df.to_csv('df.csv')